# Computational Final Project

## Setup and Imports

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances_argmin_min
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from scipy.stats import mode

from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from textblob import TextBlob


## Loading and Preprocessing the Data

In [2]:
data = pd.read_csv("../data/fake reviews dataset.csv")

data.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5,CG,Very nice set. Good quality. We have had the s...


In [3]:
data['text_length'] = data['text_'].apply(len)
data['sentiment_score'] = data['text_'].apply(lambda text: TextBlob(text).sentiment.polarity)
data['unique_word_ratio'] = data['text_'].apply(lambda x: len(set(x.split())) / len(x.split()))


In [4]:
vectorizer = TfidfVectorizer(max_features=100)
tfidf_features = vectorizer.fit_transform(data['text_']).toarray()
tfidf_df = pd.DataFrame(tfidf_features, columns=[f'tfidf_{i}' for i in range(tfidf_features.shape[1])])
df = pd.concat([data, tfidf_df], axis=1)

In [5]:
features = df[['text_length', 'rating', 'sentiment_score', 'unique_word_ratio'] + list(tfidf_df.columns)]


In [6]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [7]:
pca = PCA(n_components=5)
reduced_features = pca.fit_transform(scaled_features)


In [8]:
dbscan = DBSCAN(eps=0.8, min_samples=2)
clusters = dbscan.fit_predict(reduced_features)

In [9]:
df['cluster'] = clusters



In [10]:
# Filter out noise points (-1) assigned by DBSCAN
df = df[df['cluster'] != -1] 

In [11]:
cluster_mapping = {}
for i in set(df['cluster']):
    label_counts = df[df['cluster'] == i]['label'].value_counts()
    predominant_label = label_counts.idxmax()
    cluster_mapping[i] = predominant_label

df['predicted_label'] = df['cluster'].map(cluster_mapping)

# Confusion Matrix
conf_matrix = confusion_matrix(df['label'], df['predicted_label'], labels=['CG', 'OR'])

# Display results
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[    0 20209]
 [    0 20214]]


In [12]:
from sklearn.cluster import AgglomerativeClustering
import tqdm
agglomerative = AgglomerativeClustering(n_clusters=2)
clusters = agglomerative.fit_predict(reduced_features)

In [ ]:
clusters

In [81]:


# Map clusters to predominant labels
df['cluster'] = clusters
cluster_mapping = {}
for i in tqdm.tqdm(set(df['cluster'])):
    label_counts = df[df['cluster'] == i]['label'].value_counts()
    predominant_label = label_counts.idxmax()
    cluster_mapping[i] = predominant_label

df['predicted_label'] = df['cluster'].map(cluster_mapping)

# Confusion Matrix
conf_matrix = confusion_matrix(df['label'], df['predicted_label'], labels=['CG', 'OR'])

# Display results
print("Confusion Matrix:")
print(conf_matrix)